# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [13]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.contrib.pipeline.steps import ParallelRunStep, ParallelRunConfig

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-140495
aml-quickstarts-140495
southcentralus
f5091c60-1c3c-430f-8d81-d802f6bf2414


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [14]:
#create compute cluster
amlcompute_cluster_name="aml-compute"

#verify that cluster does not already exist
try:
    aml_compute=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_DS12_V2",max_nodes=5)
    aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [15]:
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
auto_ds_path="https://raw.githubusercontent.com/QuirkyDataScientist1978/Microsoft-Azure-Machine-Learning-Engineer-Capstone-Project-Udacity-Final-Solution-Submission/main/heart_failure_clinical_records_dataset.csv"
ds=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)

#preview data
df = ds.to_pandas_dataframe()
df.describe()

In [ ]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'heart_failure_clinical_records_dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("The dataset was found!")

else:
    url = auto_ds_path
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(ws,key)

df1 = dataset.to_pandas_dataframe()
train, test = train_test_split(df1, shuffle=True)
train.to_csv('train.csv', index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore, 'train.csv')])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
from azureml.train.automl import AutoMLConfig
import logging
import time

# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":2,
    "primary_metric":'accuracy',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":45,
    "max_concurrent_iterations":10,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    compute_target='aml-compute',
    blocked_models = ['XGBoostClassifier'],
    training_data=train,
    label_column_name='DEATH_EVENT',
    featurization = 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
    )

In [ ]:
from azureml.core.experiment import Experiment
# choose a name for experiment
experiment_name = 'automl_run'
experiment=Experiment(ws, experiment_name)

# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.
best_remote_run,best_automl_model=remote_run.get_output()
best_run_metrics=remote_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

In [ ]:
#TODO: Save the best model
print(best_remote_run)
best_remote_run

In [ ]:
#TODO: Save the best model
import pickle
best_remote_run,best_automl_model=remote_run.get_output()
#joblib.dump(value=best_automl_model,filename='automl_model.joblib')
pkl_filename = "automl_model.pkl"
with open(pkl_filename,'wb') as f:
    pickle.dump(best_automl_model,f)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model
# Register model
model = Model.register(workspace = ws,
                        model_name = "Heart_failure_prediction",
                        model_path = "./automl_model.pkl",
                        description = "Best AutoML Model",)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies


#defining an inference configuration
service_name = 'heart-failure-automl'

#setting up a curated environment
from azureml.automl.core.shared import constants
best_remote_run.download_file(constants.CONDA_ENV_FILE_PATH, 'azureml-automl.yml')
env = Environment.get(ws, name="azureml-automl")

#combining scoring script and environment in inference configuration
inference_config = InferenceConfig(entry_script='scoring.py', environment=env)

#deploy the model as a webservice
from azureml.core.webservice import LocalWebservice, Webservice

#set deployment configuration
local_deployment_config = LocalWebservice.deploy_configuration(port=8000)
service = Model.deploy(ws, "heart-failure-automl", [model], inference_config, local_deployment_config)

print(service.state)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True) #service.update

service = Model.deploy(workspace=ws,
                        name='heart-failure-automl',
                        models=[model],
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
#retrieve deployed model uri
print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
#retrive primary key
primary, secondary= service.get_keys()
print(primary)

In [ ]:
#Testing of the deployed webservice
import requests
import json
import pandas as pd
import numpy as np

# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back
data = {"data":
        [
            {
               "age":55,
               "anaemia":0,
               "creatinine_phosphokinase":7861,
               "diabetes":0,
               "ejection_fraction":38,
               "high_blood_pressure":0,
               "platelets":263358.03,
               "serum_creatinine":1.1,
               "serum_sodium":136,
               "sex":1,
               "smoking":0,
               "time":6
               },
            {
               "age":60,
               "anaemia":0,
               "creatinine_phosphokinase":68,
               "diabetes":0,
               "ejection_fraction":20,
               "high_blood_pressure":0,
               "platelets":119000,
               "serum_creatinine":2.9,
               "serum_sodium":127,
               "sex":1,
               "smoking":1,
               "time":64
                }
        ]
        }
# data = {"data":
#         [
#             [55, 0, 7861, 0, 38, 0, 263358.03, 1.1, 136, 1, 0, 6],
#             [60, 0, 68, 0, 20, 0, 119000, 2.9, 127, 1, 1, 64]
               
#         ]
#         }

#d=pd.DataFrame(data)
#d=d.to_dict()
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {primary}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#logs of the webservice
logs=service.get_logs()
print(logs)

In [ ]:
#delete the deployed webservice
service.delete()